In [1]:
import cobra
import os

In [2]:
data_directory = os.getcwd() + "/raw_data/"


# Load Models and inspect them

### Core Model

In [3]:
core_model = cobra.io.read_sbml_model(data_directory + "e_coli_core.xml")
core_model.summary()

Metabolite,Reaction,Flux,C-Number,C-Flux
glc__D_e,EX_glc__D_e,10,6,100.00%
nh4_e,EX_nh4_e,4.765,0,0.00%
o2_e,EX_o2_e,21.8,0,0.00%
pi_e,EX_pi_e,3.215,0,0.00%
Metabolite,Reaction,Flux,C-Number,C-Flux
co2_e,EX_co2_e,-22.81,1,100.00%
h2o_e,EX_h2o_e,-29.18,0,0.00%
h_e,EX_h_e,-17.53,0,0.00%


In [4]:
iAf1260_model = cobra.io.read_sbml_model(data_directory + "iAF1260.xml")
iAf1260_model.summary()

Metabolite,Reaction,Flux,C-Number,C-Flux
ca2_e,EX_ca2_e,0.00349,0,0.00%
cl_e,EX_cl_e,0.00349,0,0.00%
cobalt2_e,EX_cobalt2_e,0.002327,0,0.00%
cu2_e,EX_cu2_e,0.002327,0,0.00%
fe2_e,EX_fe2_e,0.0108,0,0.00%
glc__D_e,EX_glc__D_e,8,6,100.00%
k_e,EX_k_e,0.1308,0,0.00%
mg2_e,EX_mg2_e,0.005816,0,0.00%
mn2_e,EX_mn2_e,0.002327,0,0.00%
mobd_e,EX_mobd_e,0.002327,0,0.00%


### Helper functions

In [5]:
def get_EX_reactions_list(model):
    #Print all exchange reactions IDs & add IDs to EXCHANGE_list
    ex_reactions = []

    for reaction in model.reactions:
        if str(reaction.id).startswith("EX_"):
            ex_reactions.append(reaction.id)

    return ex_reactions


def print_EX_lower_bounds(model):
    for reaction in model.reactions:
        if str(reaction.id).startswith("EX_"):
            print(f"{reaction.id}, {reaction.lower_bound}")

            
def set_all_lower_bounds(model, lower_bound = -1000):
    for reaction in model.reactions:
        if str(reaction.id).startswith("EX_"):
            reaction.lower_bound = -1000


def print_essential_A(metabolites):
    #metabolites is a dict with metabolite:objective_value (for this metaboite being knocked out)
    for k, v in metabolites.items():
        if v <= 0:
            print(f"{k}: {v}")

#metabolites: dictionary with string of reaction id as key, lower_bound as value.
# Contains a set of lower_bounds for all EX_ reactions, that lead to a biomass growth of zero (obejctive value)
#Prints all Metabolites that are knocked out, when using appraoch B (in alphabetical order):
def print_essential_B(metabolites, print_len = False):
    i = 0
    sorted_dict = {key: value for key, value in sorted(metabolites.items())}
    for k, v in sorted_dict.items():
        #print(f"{k} ({v})")

        if v != 0:
            print(f"{k} ({v})")
            i += 1

    if print_len:
        print(i)



### Set Objective reactions 
=Biomass producing reaction
(is set to this by default already)

In [6]:
core_model.objective = "BIOMASS_Ecoli_core_w_GAM"
iAf1260_model.objective = "BIOMASS_Ec_iAF1260_core_59p81M"

In [7]:
#Show objective reaction Info
core_model.reactions.get_by_id("BIOMASS_Ecoli_core_w_GAM") #get id either from 'core_model', 'core_model.summary()' or from list of all reactions

Reaction identifier,BIOMASS_Ecoli_core_w_GAM
Name,Biomass Objective Function with GAM
Memory address,0x7f622174c700
Stoichiometry,1.496 3pg_c + 3.7478 accoa_c + 59.81 atp_c + 0.361 e4p_c + 0.0709 f6p_c + 0.129 g3p_c + 0.205 g6p_c + 0.2557 gln__L_c + 4.9414 glu__L_c + 59.81 h2o_c + 3.547 nad_c + 13.0279 nadph_c + 1.7867 oaa_c... 1.496 3-Phospho-D-glycerate + 3.7478 Acetyl-CoA + 59.81 ATP C10H12N5O13P3 + 0.361 D-Erythrose 4-phosphate + 0.0709 D-Fructose 6-phosphate + 0.129 Glyceraldehyde 3-phosphate + 0.205 D-Glucose...
GPR,
Lower bound,0.0
Upper bound,1000.0


# Original State FBA

In [8]:
core_solution = core_model.slim_optimize()
iAf1260_solution = iAf1260_model.slim_optimize()

print(f"core: {core_solution},\niAf1260: {iAf1260_solution}")

core: 0.8739215069684271,
iAf1260: 0.7367009388648729


# Set lower bounds to -1000
Set all exchange reaction's intake to -1000

In [9]:
set_all_lower_bounds(core_model)
set_all_lower_bounds(iAf1260_model)

In [10]:
core_solution = core_model.slim_optimize()
iAf1260_solution = iAf1260_model.slim_optimize()

print(f"core: {core_solution},\niAf1260: {iAf1260_solution}")

core: 40.85486846759244,
iAf1260: 3748.822043475117


# Approach A
Start with all uptakes enabled,  
switch off one uptake and check if growth is  still possible.  
Switch the uptake back on and test the next component.

In [11]:

def approach_A(model, lower_value = 0, initial_lower_value = -1000, rounding_digit = 5):
    #returns dictionary with changed reaction id as key, corresponding objective value as value
    solutions = {}
    
    EX_reactions = get_EX_reactions_list(model)

    for reaction in EX_reactions:

        model.reactions.get_by_id(reaction).lower_bound = lower_value
        objective_value = model.slim_optimize() #FBA

        #if round(objective_value, rounding_digit) <= 0:
        solutions[reaction] = round(objective_value, rounding_digit)
        
        #reset lower bound to value of parameter "initial_lower_value"
        model.reactions.get_by_id(reaction).lower_bound = initial_lower_value

    return solutions




In [66]:
core_A = approach_A(core_model)

print(core_A)

{'EX_ac_e': 40.85487, 'EX_acald_e': 40.85487, 'EX_akg_e': 40.85487, 'EX_co2_e': 40.85487, 'EX_etoh_e': 40.85487, 'EX_for_e': 40.85487, 'EX_fru_e': 40.85487, 'EX_fum_e': 40.85487, 'EX_glc__D_e': 40.85487, 'EX_gln__L_e': 40.85487, 'EX_glu__L_e': 40.85487, 'EX_h_e': 40.85487, 'EX_h2o_e': 40.85487, 'EX_lac__D_e': 40.85487, 'EX_mal__L_e': 40.85487, 'EX_nh4_e': 40.85487, 'EX_o2_e': 28.43733, 'EX_pi_e': -0.0, 'EX_pyr_e': 40.85487, 'EX_succ_e': 40.85487}


In [13]:
iAf1260_A = approach_A(iAf1260_model)
#print_EX_lower_bounds(iAf1260_model)

print_essential_A(iAf1260_A)


EX_ca2_e: -0.0
EX_cl_e: 0.0
EX_cobalt2_e: 0.0
EX_mg2_e: -0.0
EX_mn2_e: -0.0
EX_mobd_e: 0.0
EX_zn2_e: 0.0
EX_k_e: -0.0


In [71]:
def use_only_essential(model, solution_dict, lower_value = -1000):
    EX_reactions = get_EX_reactions_list(model)
    essential_list = []
    for r, v in solution_dict.items():
        if v <= 0:
            essential_list.append(r)

    for reaction in EX_reactions:
        if reaction in essential_list:
            model.reactions.get_by_id(reaction).lower_bound = lower_value
        else:
            model.reactions.get_by_id(reaction).lower_bound = 0

    biomass_growth = model.slim_optimize()

    return biomass_growth

In [72]:
set_all_lower_bounds(core_model)
core_A_test = use_only_essential(core_model, core_A)
print(core_A_test)



nan


In [73]:
set_all_lower_bounds(iAf1260_model)
iAf1260_A_test = use_only_essential(iAf1260_model, iAf1260_A)
print(iAf1260_A_test)


nan


### Approach A interpretation:

For this approach, there was always only a single reaction turned off.
If the FBA leads to an objective value of zero, then the microorganism cant grow without this metabolite.
Setting the others to zero may have an impact on growth rate/biomass production, O2, but the organism can switch to fermentation to combat the missing electron acceptor.

This mean, for the core model, only Pi is an essential Metabolite.
For the iAf1260 model, Ca2+, Cl-, Cobalt2+, Mg2+, Mn2+, Mobd, Zn2+, k are all essential.
If just one of them is missing, there wont be any biomass growth.

#### Does enabling all uptakes identified as essential (and  switching off the uptake for all other exchange reactions) lead to a valid growth medium?  
No, as the essential mean, that they have to be there, otherwise it wont work. This can not be turned around, to say, that only with them it would work.
#### Does this approach guarantee that you find a valid growth  medium? Why (not)?  
Yes, it will lead to a valid growth medium (within the model). But this wont necessarily be the best/minimal growth medium. Because only one reaction is restricted at a time and we always check for the FBA.
Compared to real life, some pathways might not be accounted for.
#### Is the result necessarily unique? If not, what could it  depend on?
The result is unique, since no combinations are tested, only ever one reactions is tested/knocked out at a time.

# Approach B
Enable all uptakes, switch one off and check if growth is possible.  
    If growth is still possible, remove the next one,  
    If not, switch it back on and then remove the next one.  
Repeat until all uptakes were tested

Returns a set of metabolites, that are a set of minimum media that still allows for growth

In [236]:
from random import shuffle

#Returns a dictionary with reaction ids as keys, lower_bounds as value. 
# This shows a set of metabolites, that are essential to biomass growth 
def approach_B(model, lower_value = 0, initial_lower_value = -1000, rounding_digit = 5, randomize_order = True):
    

    solution = {}

    reactions = get_EX_reactions_list(model)

    if randomize_order:
        shuffle(reactions)

    #test uptakes by iterating over exchange reactions id:
    for reaction in reactions: #model.medium:
        model.reactions.get_by_id(reaction).lower_bound = lower_value

        if round(model.slim_optimize(), rounding_digit) > 0:
            print(reaction)
            solution[reaction] = model.reactions.get_by_id(reaction).lower_bound
            continue
        else:
            model.reactions.get_by_id(reaction).lower_bound = initial_lower_value
            
        solution[reaction] = model.reactions.get_by_id(reaction).lower_bound    
        

    return solution

In [237]:
set_all_lower_bounds(core_model)
set_all_lower_bounds(iAf1260_model)

In [252]:
core_B = approach_B(core_model, randomize_order=False)
#print(core_B)
print_essential_B(core_B, print_len=True)


EX_ac_e
EX_acald_e
EX_akg_e
EX_co2_e
EX_etoh_e
EX_for_e
EX_fru_e
EX_fum_e
EX_glc__D_e
EX_gln__L_e
EX_h_e
EX_h2o_e
EX_lac__D_e
EX_nh4_e
EX_o2_e
EX_pyr_e
EX_succ_e
EX_glu__L_e (-1000)
EX_mal__L_e (-1000)
EX_pi_e (-1000)
3


In [45]:
iAf1260_B = approach_B(iAf1260_model)

print_essential_B(iAf1260_B, print_len = True)


EX_ca2_e (-1000)
EX_cl_e (-1000)
EX_cobalt2_e (-1000)
EX_cu2_e (-1000)
EX_dgmp_e (-1000)
EX_fe3dcit_e (-1000)
EX_k_e (-1000)
EX_mg2_e (-1000)
EX_mn2_e (-1000)
EX_mobd_e (-1000)
EX_so4_e (-1000)
EX_zn2_e (-1000)
12


### Approach B interpretation
##### Does it guarantee a valid growth medium?  
Yes, as for every knockout, the objective function is checked and the metabolite intake turned on again, if there is no growth.
##### Does it guarantee a medium with a minimal number of components?  
No, because the order of knock-outs is important. At one iteration, a smaller subset might be found than at another iteration.
Therefore a randomization step was implemented, that shows on different runs different metabolites to be essential and also some times a different number of metabolites.
This might be, because some metabolites can take the same role within pathways, so can be swappable. Also some metabolites might replace more than one other metabolite.
##### Is the result necessarily unique? If not, what could it  depend on?
See above. In different iterations, the results differ. It depends on the above mentioned order of knockouts as well as the role(s) of the metabolites.

# Approach C


Set up a linear program to minimize the number of uptake reactions that enables growth

In [18]:
set_all_lower_bounds(core_model)
set_all_lower_bounds(iAf1260_model)

In [19]:
growth_rate = 1

core_C = cobra.medium.minimal_medium(core_model, growth_rate, minimize_components=True)
print(core_C)



EX_glu__L_e    332.395406
EX_o2_e        500.000000
EX_pi_e         55.795559
dtype: float64


In [20]:
iAf1260_C = cobra.medium.minimal_medium(iAf1260_model, growth_rate, minimize_components=True)
print(iAf1260_C)


EX_amp_e        1000.000000
EX_ca2_e           0.004737
EX_cl_e            0.004737
EX_cobalt2_e       0.003158
EX_cu2_e           0.003158
EX_mg2_e           0.007895
EX_mn2_e           0.003158
EX_mobd_e          0.003158
EX_fe3dcit_e    1000.000000
EX_zn2_e           0.003158
EX_so4_e           0.250250
EX_k_e             0.177600
dtype: float64


### Approach C Iterpretation

#### Does it result in the same components as strategies A and B?  
No, there are differences (see above)
#### If yes, is this necessarily the case? If no, why not?  
A and B are iterative processes, while C is a mathematical optimization approach.
#### Is the result of strategy C necessarily unique? If no, what  could it depend on?

